In [ ]:
%%capture
%pip install pandas matplotlib plotly 'nbformat>=4.2.0' 'ipywidgets>=7.6'
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from quisp import QNode, Network, ChannelOption, QNodeAddr, NativeSimulator, Config, NetworkBuilder

In [ ]:
n = NetworkBuilder("simple_7nodes_internetworking")
n.add_qnode([QNodeAddr(1,1), QNodeAddr(3,1)], is_initiator=True)
n.add_qnode([QNodeAddr(3,2)])
n.add_qnode([QNodeAddr(3,3)])
n.add_qnode([QNodeAddr(1,99),QNodeAddr(3,99)])
n.add_qnode([QNodeAddr(2,99),QNodeAddr(4,99)])
n.add_qnode([QNodeAddr(4,2)])
n.add_qnode([QNodeAddr(2,1), QNodeAddr(4,1)], is_resipient=True)
n.connect_linear()
network = n.network

sim = NativeSimulator("../quisp")
sim.load(network)
await sim.run()
print(sim.error_messages)
sim.read_result()
print(sim.results)
print(sim.df)
sim.df[sim.df['msg_type'] == 'ConnectionSetupResponse']

In [ ]:
network = Network("test")
qnode1 = QNode(network, name="qnode1", addr=QNodeAddr(1,1), is_initiator=True)
qnode2 = QNode(network, name="qnode2", addr=QNodeAddr(1,2))
qnode3 = QNode(network, name="qnode3", addr=QNodeAddr(1,3))
qnode4 = QNode(network, name="qnode4", addr=QNodeAddr(1,4))
qnode5 = QNode(network, name="qnode5", addr=QNodeAddr(1,5))
qnode6 = QNode(network, name="qnode6", addr=QNodeAddr(1,6))
qnode7 = QNode(network, name="qnode7", addr=QNodeAddr(1,7), is_recipient=True)
qnode1.connect(qnode2, ChannelOption(quantum_channel_distance=1.0, classical_channel_distance=1.0))
qnode2.connect(qnode3, ChannelOption(quantum_channel_distance=1.0, classical_channel_distance=1.0))
qnode3.connect(qnode4, ChannelOption(quantum_channel_distance=1.0, classical_channel_distance=1.0))
qnode4.connect(qnode5, ChannelOption(quantum_channel_distance=1.0, classical_channel_distance=1.0))
qnode5.connect(qnode6, ChannelOption(quantum_channel_distance=1.0, classical_channel_distance=1.0))
qnode6.connect(qnode7, ChannelOption(quantum_channel_distance=1.0, classical_channel_distance=1.0))


sim = NativeSimulator("../quisp")
sim.load(network)
await sim.run()
sim.read_result()

In [ ]:
# sim.df.head(20)
print(sim.output)